In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import vllm

In [ ]:
# Load the model and tokenizer
MODEL_NAME = "deepseek-ai/deepseek-math-7b-rl"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, device_map="auto")

In [ ]:
# LoRA Configuration
lora_config = LoraConfig(
    r=8,  # LoRA rank
    lora_alpha=16,  # Alpha scaling
    lora_dropout=0.1,  # Dropout probability
    bias="none"")